In [17]:
import re
import random
import os

from tqdm import tqdm
import pandas as pd
import torch

from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForTokenClassification, RobertaTokenizerFast, get_scheduler
from transformers import DataCollatorForTokenClassification
from datasets import Dataset, DatasetDict, Features, Sequence, Value, ClassLabel
from accelerate import Accelerator

from sklearn.metrics import precision_recall_fscore_support

In [2]:
# Reduce VRAM usage by reducing fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


## Combine all .txt and .ann files and combine them per medicine

In [3]:
# Base folders containing annotation and text files
annotations_folder = 'annotations/'
original_texts_folder = 'originaltexts/'
output_folder = 'output_datasets/'
os.makedirs(output_folder, exist_ok=True)

# Group files by medicine
file_groups = {}
for file_name in os.listdir(annotations_folder):
    if file_name.endswith('.ann'):
        base_name = '.'.join(file_name.split('.')[:-1])
        medicine = base_name.rsplit('.', 1)[0]
        file_groups.setdefault(medicine, []).append(file_name)

# Process each group
for medicine, ann_files in file_groups.items():
    combined_output = []

    for ann_file in ann_files:
        txt_file = ann_file.replace('.ann', '.txt')
        txt_path = os.path.join(original_texts_folder, txt_file)
        ann_path = os.path.join(annotations_folder, ann_file)

        # Ensure the corresponding .txt file exists
        if not os.path.exists(txt_path):
            raise FileNotFoundError(f"Text file not found for annotation file {ann_file}")

        # Read the content of the .ann and .txt files
        with open(ann_path, 'r') as ann_f:
            ann_lines = ann_f.readlines()

        with open(txt_path, 'r') as txt_f:
            txt_content = txt_f.read()

        # Parse annotations and filter out AnnotatorNotes
        annotations = []
        for line in ann_lines:
            if line.startswith('T'):
                parts = line.strip().split('\t')
                if len(parts) == 3:
                    tag_info, word = parts[1], parts[2]
                    tag_parts = tag_info.split()
                    if len(tag_parts) >= 3:
                        tag = tag_parts[0]
                        try:
                            start_idx = int(tag_parts[1])
                            end_idx = int(tag_parts[2])
                        except ValueError:
                            if ';' in tag_parts[2]:  # Handle ranges like '742;763'
                                start_idx = int(tag_parts[1])
                                end_idx = int(tag_parts[2].split(';')[-1])
                            else:
                                raise ValueError(f"Unexpected annotation format: {tag_parts}")
                        annotations.append((start_idx, end_idx, tag, word))

        # Sort annotations by start index
        annotations.sort(key=lambda x: x[0])

        # Generate output format
        output = []
        current_idx = 0
        for start_idx, end_idx, tag, word in annotations:
            # Add text between the last annotation and the current annotation as "O"
            if current_idx < start_idx:
                intervening_text = txt_content[current_idx:start_idx]
                for token in re.findall(r"\w+(?:'\w+)?|[.,!?]", intervening_text):
                    output.append(f"{token} O")

            # Add the annotated word with its tag
            for i, token in enumerate(word.split()):
                tag_prefix = 'B-' if i == 0 else 'I-'
                output.append(f"{token} {tag_prefix}{tag}")

            current_idx = end_idx

        # Add remaining text as "O"
        if current_idx < len(txt_content):
            remaining_text = txt_content[current_idx:]
            for token in re.findall(r"\w+(?:'\w+)?|[.,!?]", remaining_text):
                output.append(f"{token} O")

        # Add to combined output with a newline separator
        combined_output.extend(output)
        combined_output.append('')  # Empty line between posts

    # Write combined output to file
    combined_output_text = '\n'.join(combined_output).strip()
    output_file = os.path.join(output_folder, f"{medicine}_combined_output.txt")
    with open(output_file, 'w') as out_f:
        out_f.write(combined_output_text)

    print(f"Combined output saved for {medicine} in {output_file}")

Combined output saved for ARTHROTEC in output_datasets/ARTHROTEC_combined_output.txt
Combined output saved for CAMBIA in output_datasets/CAMBIA_combined_output.txt
Combined output saved for CATAFLAM in output_datasets/CATAFLAM_combined_output.txt
Combined output saved for DICLOFENAC-POTASSIUM in output_datasets/DICLOFENAC-POTASSIUM_combined_output.txt
Combined output saved for DICLOFENAC-SODIUM in output_datasets/DICLOFENAC-SODIUM_combined_output.txt
Combined output saved for FLECTOR in output_datasets/FLECTOR_combined_output.txt
Combined output saved for LIPITOR in output_datasets/LIPITOR_combined_output.txt
Combined output saved for PENNSAID in output_datasets/PENNSAID_combined_output.txt
Combined output saved for SOLARAZE in output_datasets/SOLARAZE_combined_output.txt
Combined output saved for VOLTAREN-XR in output_datasets/VOLTAREN-XR_combined_output.txt
Combined output saved for VOLTAREN in output_datasets/VOLTAREN_combined_output.txt
Combined output saved for ZIPSOR in output_da

## Combine all the medicine files into one dataset

In [4]:
# Folder containing all combined output files
output_datasets_folder = 'output_datasets/'
final_output_file = 'final_dataset.txt'

# Ensure the folder exists
if not os.path.exists(output_datasets_folder):
    raise FileNotFoundError(f"The folder {output_datasets_folder} does not exist.")

# List all files in the folder
output_files = [f for f in os.listdir(output_datasets_folder) if f.endswith('_combined_output.txt')]

# Combine all files into a single final dataset
final_dataset = []
for file_name in output_files:
    file_path = os.path.join(output_datasets_folder, file_name)
    with open(file_path, 'r') as f:
        content = f.read().strip()  # Read and strip any trailing spaces or newlines
        final_dataset.append(content)

    # Add an empty line to separate posts from different files
    final_dataset.append('')

# Write the combined dataset to the final output file
with open(final_output_file, 'w') as f:
    f.write('\n'.join(final_dataset).strip())  # Ensure no extra trailing newline

print(f"Final dataset saved to {final_output_file}")

Final dataset saved to final_dataset.txt


## Read the final dataset into the Iob dataset format

In [5]:
def read_iob_file(file_path):
    """Reads an IOB file from filepath and returns sentences with tokens and tags."""
    sentences = []
    sentence_tokens = []
    sentence_labels = []

    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            line = line.strip()
            if line:  # If line is not empty
                token, tag = line.split()
                sentence_tokens.append(token)
                sentence_labels.append(tag)

            else:
                # End of a sentence
                if sentence_tokens:
                    sentences.append({"tokens": sentence_tokens, "ner_tags": sentence_labels})
                    sentence_tokens = []
                    sentence_labels = []
        # Add the last sentence if file doesn't end with a newline
        if sentence_tokens:
            sentences.append({"tokens": sentence_tokens, "ner_tags": sentence_labels})
    return sentences

def create_dataset_from_final_file(final_file_path):
    """Create a dataset from a single IOB file and return it as a DatasetDict."""

    if not os.path.exists(final_file_path):
        raise FileNotFoundError(f"The file {final_file_path} does not exist.")

    # Parse the file
    data = read_iob_file(final_file_path)

    # Define the label names and ClassLabel feature
    unique_labels = sorted(set(tag for d in data for tag in d["ner_tags"]))
    label_feature = ClassLabel(names=unique_labels)

    # Define the Features schema for Hugging Face datasets
    features = Features({
        'tokens': Sequence(Value("string")),
        'ner_tags': Sequence(label_feature)
    })

    # Convert data into a Dataset
    dataset = Dataset.from_list(data).cast(features)

    # Create a DatasetDict
    dataset_dict = DatasetDict({"full_data": dataset})

    return dataset_dict


In [6]:
final_dataset_path = "final_dataset.txt"
dataset_dict = create_dataset_from_final_file(final_dataset_path)
dataset = dataset_dict['full_data']

Casting the dataset: 100%|██████████| 1248/1248 [00:00<00:00, 6870.58 examples/s]


## Tokenize and align labels, also add datacollator

In [7]:
# def align_labels_with_tokens(labels, word_ids):
#     new_labels = []
#     current_word = None
#
#     for word_id in word_ids:
#         if word_id != current_word:
#             current_word = word_id
#             label = -100 if word_id is None else labels[word_id]
#             new_labels.append(label)
#
#         elif word_id is None:
#             # Special token
#             new_labels.append(-100)
#
#         else:
#             # Same word as previous token
#             label = labels[word_id]
#
#             # If the label is B-XXX we change it to I-XXX
#             if label % 2 == 1:
#                 label += 1
#             new_labels.append(label)
#
#     return new_labels
#
#
# def tokenize_and_align_labels(examples):
#     tokenized_inputs = tokenizer(
#         examples["tokens"], truncation=True,
#         is_split_into_words=True
#     )
#     all_labels = examples["ner_tags"]
#     new_labels = []
#     for i, labels in enumerate(all_labels):
#         word_ids = tokenized_inputs.word_ids(i)
#         new_labels.append(align_labels_with_tokens(labels, word_ids))
#
#     tokenized_inputs["labels"] = new_labels
#     return tokenized_inputs

## Dataset generators

In [8]:
def generate_train_datasets(dataset_, number_of_samples, number_of_splits):
    """
    Generates train datasets by sampling from the given dataset based on the number of samples and splits.

    Args:
        dataset_ (Dataset): The base dataset to sample from.
        number_of_samples (int): Number of samples per dataset.
        number_of_splits (int): Number of datasets to generate (different seeds).

    Returns:
        List[Tuple[str, Dataset, List[int]]]: List of generated datasets with their names and indices.
    """
    datasets = []

    for seed in range(number_of_splits):
        # Set the random seed for reproducibility
        random.seed(seed)

        # Shuffle and sample from the dataset
        indices = list(range(len(dataset_)))
        random.shuffle(indices)
        sampled_indices = indices[:number_of_samples]

        sampled_dataset = dataset_.select(sampled_indices)

        # Add the dataset with its name and indices
        datasets.append((f"train_dataset_{number_of_samples}_{seed}", sampled_dataset, sampled_indices))

    return datasets

In [9]:
def generate_validation_datasets(dataset_, train_indices, number_of_samples, number_of_splits):
    """
    Generates validation datasets by sampling from the given dataset, ensuring no overlap with training data.

    Args:
        dataset_ (Dataset): The base dataset to sample from.
        train_indices (List[int]): Indices of the training dataset to exclude from sampling.
        number_of_samples (int): Number of samples per validation dataset.
        number_of_splits (int): Number of validation datasets to generate (different seeds).

    Returns:
        List[Tuple[str, Dataset, List[int]]]: List of generated validation datasets with names and indices.
    """
    datasets = []
    all_indices = set(range(len(dataset_)))
    available_indices = list(all_indices - set(train_indices))  # Exclude training indices

    for seed in range(number_of_splits):
        # Set the random seed for reproducibility
        random.seed(seed)

        # Shuffle and sample from the remaining indices
        random.shuffle(available_indices)
        sampled_indices = available_indices[:int(number_of_samples / 5)]

        sampled_dataset = dataset_.select(sampled_indices)

        # Add the dataset with its name and indices
        datasets.append((f"val_dataset_{number_of_samples/5}_{seed}", sampled_dataset, sampled_indices))

    return datasets

In [10]:
def generate_test_datasets(dataset_, train_indices, val_indices, number_of_samples, number_of_splits):
    """
    Generates test datasets by sampling from the given dataset, ensuring no overlap with training or validation data.

    Args:
        dataset_ (Dataset): The base dataset to sample from.
        train_indices (List[int]): Indices of the training dataset to exclude from sampling.
        val_indices (List[int]): Indices of the validation dataset to exclude from sampling.
        number_of_samples (int): Number of samples per test dataset.
        number_of_splits (int): Number of test datasets to generate (different seeds).

    Returns:
        List[Tuple[str, Dataset]]: List of generated test datasets with names.
    """
    datasets = []
    all_indices = set(range(len(dataset_)))
    available_indices = list(all_indices - set(train_indices) - set(val_indices))  # Exclude train and val indices

    for seed in range(number_of_splits):
        sampled_indices = available_indices[:]

        sampled_dataset = dataset_.select(sampled_indices)

        # Add the dataset with its name
        datasets.append((f"test_dataset_{number_of_samples}_{seed}", sampled_dataset))

    return datasets

### Example usage

In [11]:
# Step 1: Generate Train Dataset
# train_datasets = generate_train_datasets(dataset, number_of_samples=30, number_of_splits=1)
# train_name, train_dataset, train_indices = train_datasets[0]
# print(f"{train_name}: {len(train_dataset)} samples")
#
# # Step 2: Generate Validation Dataset
# val_datasets = generate_validation_datasets(dataset, train_indices, number_of_samples=30, number_of_splits=1)
# val_name, val_dataset, val_indices = val_datasets[0]
# print(f"{val_name}: {len(val_dataset)} samples")
#
# # Step 3: Generate Test Dataset
# test_datasets = generate_test_datasets(dataset, train_indices, val_indices, number_of_samples=30, number_of_splits=1)
# test_name, test_dataset = test_datasets[0]
# print(f"{test_name}: {len(test_dataset)} samples")

# Prepping for training

In [12]:
label_names = dataset.features["ner_tags"].feature.names
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [13]:
def create_dataset_given_model(tokenizer_):
    data_collator_ = DataCollatorForTokenClassification(tokenizer=tokenizer_)

    # Tokenize and align labels
    tokenized_dataset_ = dataset.map(tokenize_and_align_labels, batched=True, remove_columns=dataset.column_names)

    # Step 1: Generate Train Dataset
    train_datasets = generate_train_datasets(tokenized_dataset_, number_of_samples=100, number_of_splits=1)
    train_name, train_dataset, train_indices = train_datasets[0]
    print(f"{train_name}: {len(train_dataset)} samples")

    # Step 2: Generate Validation Dataset
    val_datasets = generate_validation_datasets(tokenized_dataset_, train_indices, number_of_samples=100, number_of_splits=1)
    val_name, val_dataset, val_indices = val_datasets[0]
    print(f"{val_name}: {len(val_dataset)} samples")

    # Step 3: Generate Test Dataset
    test_datasets = generate_test_datasets(tokenized_dataset_, train_indices, val_indices, number_of_samples=100, number_of_splits=1)
    test_name, test_dataset = test_datasets[0]
    print(f"{test_name}: {len(test_dataset)} samples")

    return data_collator_, train_dataset, val_dataset, test_dataset


In [14]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Flatten predictions and labels, removing ignored indices
    true_labels = [label for label_seq in labels for label in label_seq if label != -100]
    true_predictions = [pred for pred_seq, label_seq in zip(predictions, labels)
                        for pred, label in zip(pred_seq, label_seq) if label != -100]
    return true_labels, true_predictions


# Training and evaluation

In [15]:
def iterate_and_finetune_with_torch(
    dataset,
    file_name,
    models,
    start_size=5,
    end_size=500,
    step_size=5,
    k_splits=5,
    batch_size=8,
    learning_rate=5e-5,
    weight_decay=0.0,
    num_epochs=3,
):
    """
    Fine-tune models with varying dataset sizes and k-fold splits, saving results to Excel.

    Parameters:
    - dataset (DatasetDict): Dataset for training, validation, and testing.
    - file_name (str): Excel file to save results.
    - models (dict): Dictionary of model names and their sizes.
    - start_size (int): Starting size for training datasets.
    - end_size (int): Maximum size for training datasets.
    - step_size (int): Step size for increasing dataset sizes.
    - k_splits (int): Number of k-fold splits.
    - batch_size (int): Training batch size.
    - learning_rate (float): Learning rate for fine-tuning.
    - weight_decay (float): Weight decay for optimizer.
    - num_epochs (int): Number of training epochs.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Check or create the results file
    if os.path.exists(file_name):
        results_df = pd.read_excel(file_name)
    else:
        results_df = pd.DataFrame(columns=["Train Size", "K-Fold", "Test F1", "Model"])

    for train_size in range(start_size, end_size + 1, step_size):
        for split in range(k_splits):
            for size, model_name in models.items():
                print(f"\nFine-tuning {model_name} ({size}) with Train Size {train_size}, Split {split + 1}...")

                # Initialize tokenizer
                if size == "large":
                    tokenizer = RobertaTokenizerFast.from_pretrained("roberta-large", add_prefix_space=True)
                else:
                    tokenizer = AutoTokenizer.from_pretrained(model_name)

                data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

                # Generate datasets
                train_datasets = generate_train_datasets(
                    dataset, number_of_samples=train_size, number_of_splits=k_splits
                )
                _, train_dataset, train_indices = train_datasets[split]

                val_datasets = generate_validation_datasets(
                    dataset, train_indices=train_indices, number_of_samples=train_size, number_of_splits=k_splits
                )
                _, val_dataset, val_indices = val_datasets[split]

                test_datasets = generate_test_datasets(
                    dataset, train_indices=train_indices, val_indices=val_indices,
                    number_of_samples=train_size, number_of_splits=k_splits
                )
                _, test_dataset = test_datasets[split]

                def align_labels_with_tokens(labels_, word_ids):
                    new_labels = []
                    current_word = None

                    for word_id in word_ids:
                        if word_id != current_word:
                            current_word = word_id
                            label = -100 if word_id is None else labels_[word_id]
                            new_labels.append(label)

                        elif word_id is None:
                            # Special token
                            new_labels.append(-100)

                        else:
                            # Same word as previous token
                            label = labels_[word_id]

                            # If the label is B-XXX we change it to I-XXX
                            if label % 2 == 1:
                                label += 1
                            new_labels.append(label)

                    return new_labels


                def tokenize_and_align_labels(examples):
                    tokenized_inputs = tokenizer(
                        examples["tokens"], truncation=True,
                        is_split_into_words=True
                    )
                    all_labels = examples["ner_tags"]
                    new_labels = []
                    for i, labels_ in enumerate(all_labels):
                        word_ids = tokenized_inputs.word_ids(i)
                        new_labels.append(align_labels_with_tokens(labels_, word_ids))

                    tokenized_inputs["labels"] = new_labels
                    return tokenized_inputs

                # Tokenize datasets
                tokenized_train = train_dataset.map(tokenize_and_align_labels, batched=True,  remove_columns=dataset.column_names)
                tokenized_val = val_dataset.map(tokenize_and_align_labels, batched=True,  remove_columns=dataset.column_names)
                tokenized_test = test_dataset.map(tokenize_and_align_labels, batched=True,  remove_columns=dataset.column_names)

                train_dataloader = DataLoader(tokenized_train, batch_size=batch_size, shuffle=True, collate_fn=data_collator)
                val_dataloader = DataLoader(tokenized_val, batch_size=batch_size, collate_fn=data_collator)
                test_dataloader = DataLoader(tokenized_test, batch_size=batch_size, collate_fn=data_collator)

                # Initialize the model for token classification
                model = AutoModelForTokenClassification.from_pretrained(
                    model_name, id2label=id2label, label2id=label2id
                )

                optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
                num_training_steps = num_epochs * len(train_dataloader)
                lr_scheduler = get_scheduler(
                    "linear",
                    optimizer=optimizer,
                    num_warmup_steps=0,
                    num_training_steps=num_training_steps
                )

                accelerator = Accelerator()
                model, optimizer, train_dataloader, val_dataloader = accelerator.prepare(
                    model, optimizer, train_dataloader, val_dataloader
                )

                # Training loop
                for epoch in range(num_epochs):
                    print(f"Epoch {epoch + 1}/{num_epochs}")
                    model.train()
                    total_loss = 0
                    progress_bar = tqdm(train_dataloader, desc=f"Training Epoch {epoch+1}")
                    for batch in progress_bar:
                        batch = {k: v.to(device) for k, v in batch.items()}
                        outputs = model(**batch)
                        loss = outputs.loss
                        total_loss += loss.item()

                        accelerator.backward(loss)
                        optimizer.step()
                        lr_scheduler.step()
                        optimizer.zero_grad()
                        progress_bar.set_postfix(loss=loss.item())

                    print(f"Epoch {epoch + 1} Loss: {total_loss:.4f}")

                # Validation loop
                model.eval()
                val_predictions, val_labels = [], []
                with torch.no_grad():
                    for batch in val_dataloader:
                        batch = {k: v.to(device) for k, v in batch.items()}
                        outputs = model(**batch)
                        logits = outputs.logits
                        predictions = logits.argmax(dim=-1)
                        labels = batch["labels"]

                        predictions = accelerator.gather(predictions)
                        labels = accelerator.gather(labels)

                        flat_labels, flat_predictions = postprocess(predictions, labels)
                        val_labels.extend(flat_labels)
                        val_predictions.extend(flat_predictions)

                # Test loop
                test_predictions, test_labels = [], []
                with torch.no_grad():
                    for batch in test_dataloader:
                        batch = {k: v.to(device) for k, v in batch.items()}
                        outputs = model(**batch)
                        logits = outputs.logits
                        predictions = logits.argmax(dim=-1)
                        labels = batch["labels"]

                        predictions = accelerator.gather(predictions)
                        labels = accelerator.gather(labels)

                        flat_labels, flat_predictions = postprocess(predictions, labels)
                        test_labels.extend(flat_labels)
                        test_predictions.extend(flat_predictions)

                # Calculate test metrics
                precision, recall, f1, _ = precision_recall_fscore_support(
                    test_labels, test_predictions, average="micro"
                )
                print(f"Test Metrics: Precision={precision:.4f}, Recall={recall:.4f}, F1={f1:.4f}")

                # Append results
                new_row = pd.DataFrame(
                    [{"Train Size": train_size, "K-Fold": split + 1, "Test F1": f1, "Model": model_name}]
                )
                results_df = pd.concat([results_df, new_row], ignore_index=True)
                results_df.to_excel(file_name, index=False)

                # Cleanup
                del model, train_dataloader, val_dataloader, test_dataloader
                torch.cuda.empty_cache()

    print(f"Results saved to {file_name}")


In [19]:
# Define the models and their corresponding sizes
models = {
    "small": "bert-base-cased",
    "medium": "bert-large-cased",
    "large": "roberta-large"
}

iterate_and_finetune_with_torch(dataset=dataset, file_name='Experiments_full_labeled.xlsx', models=models, start_size=5, end_size=30, step_size=5)


Fine-tuning bert-base-cased (small) with Train Size 5, Split 1...


Map: 100%|██████████| 1242/1242 [00:00<00:00, 4988.81 examples/s]
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


Training Epoch 1: 100%|██████████| 1/1 [00:00<00:00,  3.78it/s, loss=2.41]


Epoch 1 Loss: 2.4089
Epoch 2/3


...

Training Epoch 2: 100%|██████████| 1/1 [00:04<00:00,  4.56s/it, loss=1.42]


Epoch 2 Loss: 1.4191
Epoch 3/3


Training Epoch 3: 100%|██████████| 1/1 [00:09<00:00,  9.02s/it, loss=1.15]


Epoch 3 Loss: 1.1486


KeyboardInterrupt: 